<a href="https://colab.research.google.com/github/ez-zee/Chatbots-Python/blob/main/Create_video_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from cvat_sdk.api_client import Configuration, ApiClient, models
import glob
import os

# Путь к твоим видео
video_dir = r"C:\Users\felhasznaló\Desktop\CV datasets\VIDEO ANNOTATION"
video_files = glob.glob(os.path.join(video_dir, "*.mp4"))

# Проверка, что нашли видео
if not video_files:
    raise FileNotFoundError("Видео не найдены в указанной директории.")

# Конфигурация подключения к CVAT
configuration = Configuration(
    host="http://localhost:8080",  # Обязательно: укажи порт, если ты его включал
    username="Azat",      # 🔁 Заменить на логин от CVAT
    password="********",      # 🔁 Заменить на пароль
)

with ApiClient(configuration) as client:
    print("Создаём проект...")
    project_spec = models.ProjectWriteRequest(
        name="Разметка видео Roboflow",
        labels=[
            models.PatchedLabelRequest(name="object")
        ]
    )
    project, _ = client.projects_api.create(project_spec)
    print(f"✅ Проект создан: ID {project.id}")

    print("Создаём задание...")
    task_spec = models.TaskWriteRequest(
        name="Задание: 2 видео для авторазметки",
        project_id=project.id,
    )
    task, _ = client.tasks_api.create(task_spec)
    print(f"✅ Задание создано: ID {task.id}")

    print("Загружаем видео...")
    with_files = [open(file, 'rb') for file in video_files]

    task_data = models.DataRequest(
        client_files=with_files,
        image_quality=70,
        use_zip_chunks=False,
        use_cache=False,
    )

    result, _ = client.tasks_api.create_data(
        task.id,
        data_request=task_data,
        _content_type="multipart/form-data"
    )

    for f in with_files:
        f.close()

    print("✅ Видео загружены в задание и разбиты на кадры.")
